# <span style="color:red" font-size=30>Deep learning project</span>

#### group : Mariem mazouz / safa chaari / abir barouni /ghofrane soltani / med aziz omrani / aziz tebessi

### importing libraries

In [ ]:
import pandas as pd
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.utils import shuffle           
from tqdm import tqdm
from matplotlib.pyplot import imread

### Data understanding and visualization

In [ ]:
train_path = "C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET/TRAIN"

In [ ]:
os.listdir(train_path)

In [ ]:
tree_example_dir = train_path + r'/tree/00000071.jpg'
tree_example_img = imread(tree_example_dir)
tree_example_img.shape

In [ ]:
#Check number of images
category = []
number_images = []
for cat in os.listdir(train_path):
    print("Number of " + cat + " images : " + str(len(os.listdir(train_path+'/'+cat))))
    category.append(cat)
    number_images.append(len(os.listdir(train_path+'/'+cat)))    
print("Total Number of Images : " + str(np.sum(number_images)))

In [ ]:
#Visualizing with pie chart
plt.pie(number_images, labels = category, autopct='%.0f%%')
plt.show()

In [ ]:
class_names = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

IMAGE_SIZE = (200,200)

In [ ]:
def load_data():
   
    datasets = [r'C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET/TRAIN', r'C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET/Test']
    output = []
    
    # Iterate through the training and test set.
    for dataset in datasets:
        
        images = [] 
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each Subfolder corresponding to a category  
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Image path should be obtained
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv.imread(img_path)
                image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
                image = cv.resize(image, IMAGE_SIZE) 
                
                # Append the image along with its label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')
        
        # Shuffle the images to introduce some randomness in our data
        images, labels = shuffle(images, labels)
        
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
_, train_counts = np.unique(train_labels, return_counts=True)
_, test_counts = np.unique(test_labels, return_counts=True)
pd.DataFrame({'train': train_counts,'test': test_counts}, index=class_names).plot.bar()
plt.show()

### Data preparation

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
%matplotlib inline

In [ ]:
def removeCorruptedImages(path):
    for filename in os.listdir(path):
        try:
            img = Image.open(os.path.join(path,filename))
            img.verify() 
        except (IOError, SyntaxError) as e:
            print('Bad file:', filename)
            os.remove(os.path.join(path,filename))

In [ ]:
# check for any corrupted images and delete them
# code obtained from https://stackoverflow.com/questions/67505710/pil-unidentifiedimageerror-cannot-identify-image-file-io-bytesio-object
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

path = Path(train_path).rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
        print(img_p)

In [ ]:
#Explore the average dimension of the images
error_image = []
for cat in os.listdir(train_path):
    dim1 = []
    dim2 = []
    for image_filename in os.listdir(train_path+'/'+cat+'/'):    
        try:
            img = imread(train_path+'/'+cat+'/'+image_filename)
        except:
            print("Error reading file on " + cat + " %s" %image_filename)
            error_image.append(train_path +'/'+ cat + '/' + image_filename)
            continue
        if len(img.shape)==2: #Reshape some images with single color channel
            img = img.reshape(img.shape[0],img.shape[1],1)
        if len(img.shape)==0:
            print("Image shape = 0 on " + cat + " %s" %image_filename)
            error_image.append(train_path + '/'+cat + '/' + image_filename)
        else:
            d1,d2,colors = img.shape
            dim1.append(d1)
            dim2.append(d2)
    p = sns.jointplot(dim1,dim2)
    p.fig.suptitle("Dimensions of %s images" %cat)
    print("Mean of dim1 on " + cat + " is "+ str(np.mean(dim1)))
    print("Mean of dim2 on " + cat + " is "+ str(np.mean(dim2)))



In [ ]:
error_image

In [ ]:
#We'll remove all files that show problem and resize the image to 600 x 600 pixels in image generating process.
for file in error_image:
    os.remove(file)
print("Completed removing the files that have problems...")

### Data preprocessing

In [ ]:
train_path = r'C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET/TRAIN'
test_path = r'C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET/Test'

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

def preprocess_images(dataset_path):
    images_data = []
    images_label = []
    class_names = os.listdir(dataset_path)
    for class_name in class_names:
        images_path = dataset_path + '/' + class_name
        images = os.listdir(images_path)
        for image in images:
            bgr_img = cv2.imread(images_path + '/' + image)
            # dsize
            dsize = (200,200)
            #resize image
            resized_image = cv2.resize(bgr_img,dsize)
            # convert from BGR color-space to YCrCb for the luminosity correction to red and blue
            ycrcb_img = cv2.cvtColor(resized_image, cv2.COLOR_BGR2YCrCb)
            # create a CLAHE object 
            clahe = cv2.createCLAHE(clipLimit=25.0, tileGridSize=(20,20))
            # Now apply CLAHE object on the YCrCb image
            ycrcb_img[:, :, 0] = clahe.apply(ycrcb_img[:, :, 0])
            # convert back to BGR color-space from YCrCb
            equalized_img = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)
            # Denoise is done to remove unwanted noise to better perform
            equalized_denoised_image = cv2.fastNlMeansDenoisingColored(equalized_img, 10, 10, 10, 7, 21)

            images_data.append(equalized_denoised_image/255)
            images_label.append(class_name)
    #pour pouvoir les utiliser dans des opérations mathématiques et de traitement d'image ultérieures plus facilement et efficacement.
    images_data = np.array(images_data)
    images_label = np.array(images_label)
    return images_data, images_label

def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Augmented image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)
    plt.show()

train_images_data, train_images_label = preprocess_images(train_path)

# Visualize the first image in the dataset
visualize(train_images_data[10], train_images_data[11])


In [ ]:
def encoding_targets(labels):
    le = preprocessing.LabelEncoder()
    images_label = le.fit_transform(labels)
    return images_label

In [ ]:
from sklearn import preprocessing
class_names = os.listdir(train_path)
class_num = len(class_names)
train_images_label = encoding_targets(train_images_label)

## Data augmentation

In [ ]:
# Data augmentation is the process of transforming images to create new ones, for training models.
# Data augmentation increases the number of examples in the training set while also introducing more variety 
#in what the model sees and learns from.

In [ ]:
# directory with its sub folders
from os import walk
for (dirpath, dirnames, filenames) in walk(r'C:/Users/Abir/Desktop/4DS/lab_4ds_yoga/YOGA/content/cleaned/DATASET'):
    print("Directory path: ", dirpath)

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

In [ ]:
import os
import matplotlib.image as mpimg

# Create an empty list to store the image filenames
image_files = []

for file in os.listdir(dirpath):
    # Check if the file has a valid image extension
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
        # Add the file name to the list of image files
        image_files.append(os.path.join(dirpath, file))

# Print the list of image filenames
print(image_files)

# visualization: 
# Loop through each image file and display it
for image_file in image_files:
    # Load the image using matplotlib
    image = mpimg.imread(image_file)

    # Display the image using matplotlib
    plt.imshow(image)
    plt.show()

ROTATING THE IMAGE:

One of the most commonly used augmentation techniques is image rotation.
Even if we rotate the image, the information on the image remains the same.
Otherwise, A yoga pose is yogaa pose, even if we see it from a different angle.



In [ ]:
from skimage.transform import rotate
#Rotating the image to 90 degree angle
rotated_image = tf.image.rot90(image)
visualize(image, rotated_image)

#Rotating the image to 180 degree angle
rotated = rotate(image, angle=180, mode = 'wrap')
visualize(image, rotated)

APPLYING THE GRAYSCALE FEATURES TO THE IMAGE:

Grayscale augmentation randomly causes an input image to be converted to a single channel, grayscale output image

In [ ]:
grayscaled_image = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled_image))
plt.colorbar()

### Modeling phase with preprocessing 

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(200, 200, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images_data, train_images_label, batch_size = 28, epochs=5, validation_split = 0.3)

In [ ]:
def plot_performance(history):

    fig = plt.figure(figsize=(15,8))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("Training_accuracy vs Validation_accuracy")
    plt.ylabel("ACCURACY")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss_function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("Training_loss vs Validation_loss")
    plt.ylabel("LOSS")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_performance(history)

In [ ]:
test_loss = model.evaluate(train_images_data, train_images_label)

In [ ]:
#By default, the index is into the flattened array, otherwise along the specified axis.
predictions = model.predict(train_images_data)
pred_labels = np.argmax(predictions,axis=1)  # np.argmax is used since each prediction would be an array of...
                                             # probabilities and we need to pick the max value. 
pred_labels

In [ ]:
fig, ax = plt.subplots(5,5, figsize = (15,15))
ax = ax.ravel()

for i in range(0,25):  
    ax[i].imshow(train_images_data[i])
    ax[i].set_title(f"predicted class: {class_names[pred_labels[i]]} \n Actual Class: {class_names[train_images_label[i]]}")
    ax[i].axis('off')
plt.subplots_adjust(wspace=0.65)

## Modeling phase without preprocessing

### Model 2

In [ ]:
# we'll start by viewing an example of the data and normalizing between [0,1]the values since they are in the range of [0,255]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
from tensorflow.keras import regularizers
model_l2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(200, 200)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(64, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model_l2 = Sequential()
model_l2.add(Conv2D(32, (3, 3), input_shape=(200, 200, 3)))
model_l2.add(Activation('relu'))
model_l2.add(MaxPooling2D(pool_size=(2, 2)))

model_l2.add(Conv2D(64, (3, 3)))
model_l2.add(Activation('relu'))
model_l2.add(MaxPooling2D(pool_size=(2, 2)))

model_l2.add(Conv2D(128, (3, 3)))
model_l2.add(Activation('relu'))
model_l2.add(MaxPooling2D(pool_size=(2, 2)))

model_l2.add(Conv2D(128, (3, 3)))
model_l2.add(Activation('relu'))
model_l2.add(MaxPooling2D(pool_size=(2, 2)))


model_l2.add(Flatten())
model_l2.add(Dense(64))
model_l2.add(Activation('relu'))
model_l2.add(Dropout(0.5))
model_l2.add(Dense(5))
model_l2.add(Activation('softmax'))
model_l2.summary()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import cv2
import os
import tensorflow as tf
#import shap
import seaborn as sns
from sklearn import preprocessing
import tensorflow.keras.layers as tfl
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
epochs = 5
learning_rate = 0.001
from tensorflow.keras.optimizers import schedules

#defines the rate of which an algo converges to a solution
initial_learning_rate = 0.001
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)
opt = Adam(learning_rate=lr_schedule)
# compile the model with the optimizer instance
model_l2.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# we couldn't change the loss function because  the difference between the two is that categorical_crossentropy requires the true class labels to be one-hot encoded, whereas sparse_categorical_crossentropy does not. Instead, sparse_categorical_crossentropy can handle integer class labels directly, making it a more memory-efficient option for large datasets.

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True,random_state=2)

In [ ]:
# Define per-fold score containers
val_acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
acc_per_fold = []

# K-fold Cross Validation model evaluation
fold_no = 1
for train, valid in kfold.split(train_images,train_labels):
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    history = model_l2.fit(train_images[train], train_labels[train], batch_size=16, 
                        epochs=epochs, validation_data=(train_images[valid], train_labels[valid]))
    val_acc_per_fold.append(history.history['val_accuracy'])
    acc_per_fold.append(history.history['accuracy'])
    val_loss_per_fold.append(history.history['val_loss'])
    loss_per_fold.append(history.history['loss'])
    # Increase fold number
    fold_no += 1

In [ ]:
test_loss = model_l2.evaluate(test_images, test_labels)

In [ ]:
def plot_performance(history):

    fig = plt.figure(figsize=(15,8))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("Training_accuracy vs Validation_accuracy")
    plt.ylabel("ACCURACY")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss_function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("Training_loss vs Validation_loss")
    plt.ylabel("LOSS")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_performance(history)

In [ ]:
#By default, the index is into the flattened array, otherwise along the specified axis.
predictions = model_l2.predict(test_images)
pred_labels = np.argmax(predictions,axis=1)  # np.argmax is used since each prediction would be an array of...
                                             # probabilities and we need to pick the max value. 
pred_labels

In [ ]:
fig, ax = plt.subplots(5,5, figsize = (15,15))
ax = ax.ravel()

for i in range(0,25):  
    ax[i].imshow(test_images[i])
    ax[i].set_title(f"predicted class: {class_names[pred_labels[i]]} \n Actual Class: {class_names[test_labels[i]]}")
    ax[i].axis('off')
plt.subplots_adjust(wspace=0.65)

### model 3 

In [ ]:
model3 = tf.keras.Sequential([
        tfl.Conv2D(filters=16, kernel_size=(3,3), activation='relu',input_shape=(200,200,3)),
        tfl.MaxPool2D(pool_size=(2,2)),
        tfl.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
        tfl.BatchNormalization(axis=-1),
        tfl.Dropout(rate=0.25),
        
    
        tfl.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        tfl.MaxPool2D(pool_size=(2,2)),
        tfl.BatchNormalization(axis=-1),
        tfl.Dropout(rate=0.25),    
        
    
        tfl.Flatten(),
        tfl.Dense(512,activation='relu'),
        tfl.BatchNormalization(),
        tfl.Dropout(rate=0.5),
    
        tfl.Dense(class_num, activation='softmax')
        
])
model3.summary()

In [ ]:
#An epoch is when all the training data is used at once and is defined as the total number of iterations of all the training
#data in one cycle for training the machine learning model.
epochs = 5
#defines the rate of which an algo converges to a solution
learning_rate = 0.001
from tensorflow.keras.optimizers import schedules

initial_learning_rate = 0.001
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)
opt = Adam(learning_rate=lr_schedule)
# compile the model with the optimizer instance
model3.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# we couldn't change the loss function because  the difference between the two is that categorical_crossentropy requires the true class labels to be one-hot encoded, whereas sparse_categorical_crossentropy does not. Instead, sparse_categorical_crossentropy can handle integer class labels directly, making it a more memory-efficient option for large datasets.

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True,random_state=2)

In [ ]:
from keras.callbacks import EarlyStopping
# Define per-fold score containers
val_acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
acc_per_fold = []

early_stop = EarlyStopping(monitor='val_loss', patience=3)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, valid in kfold.split(train_images_data, train_images_label):
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    history = model3.fit(train_images_data[train], train_images_label[train], batch_size=16, 
                        epochs=epochs, validation_data=(train_images_data[valid], train_images_label[valid]),callbacks=[early_stop])
    val_acc_per_fold.append(history.history['val_accuracy'])
    acc_per_fold.append(history.history['accuracy'])
    val_loss_per_fold.append(history.history['val_loss'])
    loss_per_fold.append(history.history['loss'])
    # Increase fold number
    fold_no += 1

In [ ]:
# Calculate mean and standard deviation of metrics per fold
mean_acc_per_fold = [np.mean(scores) * 100 for scores in acc_per_fold]
std_acc_per_fold = [np.std(scores) for scores in acc_per_fold]

mean_val_acc_per_fold = [np.mean(scores) * 100 for scores in val_acc_per_fold]
std_val_acc_per_fold = [np.std(scores) for scores in val_acc_per_fold]

mean_loss_per_fold = [np.mean(scores) for scores in loss_per_fold]
std_loss_per_fold = [np.std(scores) for scores in loss_per_fold]

mean_val_loss_per_fold = [np.mean(scores) for scores in val_loss_per_fold]
std_val_loss_per_fold = [np.std(scores) for scores in val_loss_per_fold]

# Print results
for i in range(0, len(mean_acc_per_fold)):
    print(f'> Fold {i+1} - Training Accuracy: {mean_acc_per_fold[i]:.2f}% (+- {std_acc_per_fold[i]:.2f})')
    print(f'> Fold {i+1} - Validation Accuracy: {mean_val_acc_per_fold[i]:.2f}% (+- {std_val_acc_per_fold[i]:.2f})')
    print(f'> Fold {i+1} - Training Loss: {mean_loss_per_fold[i]:.4f} (+- {std_loss_per_fold[i]:.4f})')
    print(f'> Fold {i+1} - Validation Loss: {mean_val_loss_per_fold[i]:.4f} (+- {std_val_loss_per_fold[i]:.4f})')

# Print mean and standard deviation of metrics across all folds
print(f'> Mean Training Accuracy: {np.mean(mean_acc_per_fold):.2f}% (+- {np.mean(std_acc_per_fold):.2f})')
print(f'> Mean Validation Accuracy: {np.mean(mean_val_acc_per_fold):.2f}% (+- {np.mean(std_val_acc_per_fold):.2f})')
print(f'> Mean Training Loss: {np.mean(mean_loss_per_fold):.4f} (+- {np.mean(std_loss_per_fold):.4f})')
print(f'> Mean Validation Loss: {np.mean(mean_val_loss_per_fold):.4f} (+- {np.mean(std_val_loss_per_fold):.4f})')

In [ ]:
test_loss = model3.evaluate(test_images, test_labels)

In [ ]:
def plot_performance(history):

    fig = plt.figure(figsize=(15,8))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("Training_accuracy vs Validation_accuracy")
    plt.ylabel("ACCURACY")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss_function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("Training_loss vs Validation_loss")
    plt.ylabel("LOSS")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_performance(history)

### Model prediction : we focused on the second model based on its accuracy

In [ ]:
predictions = model3.predict(test_images)
pred_labels = np.argmax(predictions,axis=1)  # np.argmax is used since each prediction would be an array of...
                                             # probabilities and we need to pick the max value. 
pred_labels

In [ ]:
fig, ax = plt.subplots(5,5, figsize = (15,15))
ax = ax.ravel()

for i in range(0,25):  
    ax[i].imshow(test_images[i])
    ax[i].set_title(f"predicted class: {class_names[pred_labels[i]]} \n Actual Class: {class_names[test_labels[i]]}")
    ax[i].axis('off')
plt.subplots_adjust(wspace=0.65)